In [10]:
import sys
import pickle
import cv2
sys.path.append('..')    
sys.path.append('../letter_recognition')    
from letter_extraction import CompositeExtractor, ContoursExtractor, EdgeParams
from letter_extraction import ThresholdDetector, EdgeDetector, ExtractFragmentParams
from metrics import KeyboardLayout
from models import BaseDenseNet
from models import ConvDenseNet
from metrics import KeyboardLayout
from core import ImageGenerator
from utils.plot import *
from utils.model import *
from layout_estimation import TripleLayoutEstimator, LayoutUtils
from PIL import Image, ImageDraw

%matplotlib inline

In [8]:
def test_model(model, fragment_list):
    image_list = map(lambda x: x[1], fragment_list)
    image_data = np.array(image_list, dtype=np.float64).reshape(-1, 1, 32, 32)
    output_data = get_output_data(model, image_data)
    pred_list = map(lambda i: {
        'image': image_list[i], 
        'output': output_data[i], 
        'coord': fragment_list[i][0]
    }, range(len(fragment_list)))
    letter_preds_map = get_all_above_threshold(pred_list, 0.99)
    image_data = [pred['image'] for sublist in letter_preds_map.values() for pred in sublist]
    for image in image_data:
        plot_image(image)
    return letter_preds_map

model = load_base_dense_net('../letter_recognition/state_dicts/eval/in_place_m20_base_dense.pt', 20)
model.eval()

BaseDenseNet(
  (conv1): Conv2d(1, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (dense1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
  )
  (trans1): Transition(
    (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [21]:
thres_detector = ContoursExtractor(ThresholdDetector())
edge_params = EdgeParams(50, 170, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_L1, 3, 1)
extract_params = ExtractFragmentParams(150, 20, 1.7)
edge_detector = ContoursExtractor(EdgeDetector(edge_params), extract_params)
contours_extractor = CompositeExtractor([thres_detector, edge_detector])
print("Keyboard{}".format(1)) 
keyboard_layout = KeyboardLayout("../keyboards/layouts/keyboard{}.layout".format(8), '../')
fragment_list = edge_detector.get_fragment_list(keyboard_layout.get_image())
print(len(fragment_list))
letter_preds_map = test_model(model, fragment_list[3000:4000])

Keyboard1
603


RuntimeError: input has less dimensions than expected

In [ ]:
def get_middle_point(coord):
    ps, pe = coord['start_pos'], coord['end_pos']
    return ((ps[0] + pe[0]) / 2, (ps[1] + pe[1]) / 2)

def test_pair(elem1, elem2):
    middle1 = get_middle_point(elem1['coord'])
    middle2 = get_middle_point(elem2['coord'])
    middle2 = (middle2[0] + (middle2[0] - middle1[0]) * 10, middle2[1] + (middle2[1] - middle1[1]) * 10)
    image_draw = ImageDraw.Draw(pil_image)
    image_draw.line(middle1 + middle2, fill=255, width=20)
    del image_draw
    plt.figure(figsize=(14, 9))        
    plt.imshow(pil_image)
    plt.show()       

pil_image = Image.fromarray(keyboard_layout.get_image())        
#test_pair(letter_preds_map['A'][0], letter_preds_map['S'][0])    

In [ ]:
pred_list = [pred for sublist in letter_preds_map.values() for pred in sublist]
for pred in pred_list:
    pred['middle'] = np.array(get_middle_point(pred['coord']))

In [ ]:
row_preds_map = {}
for pred in pred_list:
    row_ind = LayoutUtils.get_letter_row_index(pred['letter'])
    pos_ind = LayoutUtils.get_letter_pos(pred['letter'])
    if row_ind not in row_preds_map:
        row_preds_map[row_ind] = {}
    if pos_ind not in row_preds_map[row_ind]:
        row_preds_map[row_ind][pos_ind] = []
    row_preds_map[row_ind][pos_ind].append(pred)

In [ ]:
triples_list = []

def add_triples_for_row(row_ind, pos1, pos2, row_offset):
    neigh_row = row_ind + row_offset
    if neigh_row not in row_preds_map:
        return
    for row_pred1 in row_preds_map[row_ind][pos1]:
        for row_pred2 in row_preds_map[row_ind][pos2]:
            neigh_preds = row_preds_map[neigh_row].values()
            for col_pred in [x for sublist in neigh_preds for x in sublist]:
                col_pos = LayoutUtils.get_letter_pos(col_pred['letter'])
                if col_pos > len(LayoutUtils.get_letter_rows()[row_ind]) - 1:
                    continue
                triples_list.append((row_pred1, row_pred2, col_pred))
    
for row_ind in row_preds_map.keys():
    for pos1 in row_preds_map[row_ind].keys():
        for pos2 in row_preds_map[row_ind].keys():
            if pos1 + 3 > pos2:
                continue
            add_triples_for_row(row_ind, pos1, pos2, -1)
            add_triples_for_row(row_ind, pos1, pos2, 1)

In [ ]:
def get_height_diff(layout, letter1, letter2):
    return abs(layout[letter1][1] - layout[letter2][1])

def get_col_rating(layout):
    diff_list = [('1', 'Q'), ('Q', 'A'), ('A', 'Z')]
    heights = map(lambda x: get_height_diff(layout, x[0], x[1]), diff_list)
    max_height = 0
    min_height = np.inf
    for i in range(len(heights)):
        for j in range(i + 1, len(heights)):
            max_height = max(max_height, abs(heights[i] - heights[j]))
            min_height = min(min_height, abs(heights[i] - heights[j]))
    return (float(min_height) / max_height)
    
def rate_layout(layout):
    letter_pred_map = {}
    for pred in pred_list:
        letter_coord = tuple(layout[pred['letter']])
        start_pos, end_pos = pred['coord']['start_pos'], pred['coord']['end_pos']
        if letter_coord not in letter_pred_map:
            letter_pred_map[letter_coord] = 0
        if KeyboardLayout._is_point_in_middle(letter_coord, start_pos, end_pos):
            letter_pred_map[letter_coord] = max(letter_pred_map[letter_coord], pred['pbp'])
    return get_col_rating(layout) * sum(letter_pred_map.values())

In [ ]:
best_layout = None
max_rating = 0.0
for triple in triples_list:
    row_pred1, row_pred2, col_pred = triple[0], triple[1], triple[2]
    layout_estimator = TripleLayoutEstimator(pred_list, row_pred1, row_pred2, col_pred)
    estimated_layout = layout_estimator.get_estimated_layout()
    cur_rating = rate_layout(estimated_layout)
    if cur_rating > max_rating:
        best_layout = estimated_layout
        max_rating = cur_rating
print(max_rating)

In [ ]:
pil_image = Image.fromarray(keyboard_layout.get_image())        
image_draw = ImageDraw.Draw(pil_image)
for letter, coord in best_layout.iteritems():
    image_draw.ellipse((coord[0] - 10, coord[1] - 10, coord[0] + 10, coord[1] + 10), fill=(255,0,0,255))
plt.figure(figsize=(14, 9))        
plt.imshow(pil_image)
plt.show()           